# [Vanilla Policy Gradient (VPG)](https://spinningup.openai.com/en/latest/algorithms/vpg.html)

Vanilla Policy Gradient (VPG) a.k.a REINFORCE is a roseta stone for policy gradient methods and enhanced `on-policy` algorithms (such as TRPO and PPO). VPG learns the Policy function directly, while less stable off-policy algorithms, such as DDPG and Q-learning, uses the Bellman optimality equation.

The goal in reinforcement learning is to maximize the expected cumulative reward (a.k.a expected return). In Policy Gradient Algorithms, this can be achieved by optimizing the policy parameters computing the `gradient ascent` of the performance objective and, consequently, pushing the probability distribution towards only the good actions:

\begin{eqnarray}
\theta_{t+1} &=&  \theta_t + \alpha \nabla_{\theta} J (\pi_{\theta}).\\
\end{eqnarray}

- VPG optimizes a `stochastic policy` and it is suitable for both `continuous and discrete action spaces`.

One way to reduce variance is to add a baseline to the VPG. A near-optimal choice for the baseline in the VPG is the expected return $\mathbb{E}_{\tau \sim \pi_{\theta}}[\mathcal{R}(\tau)]$ before taking the action. For any choice of baseline, the gradient estimator is unbiased. The difference between the expected return and the baseline is the advantage function (see the [rl-theory.ipynb](https://github.com/qucai-lab/rl-roadmap/blob/main/theory/rl-theory.ipynb)):

$$A^{\pi_{\theta}}(s_t, a_t) = Q^{\pi_{\theta}}(s_t, a_t) - V^{\pi_{\theta}}(s_t) \in {\rm I\!R}.$$

Where

\begin{eqnarray}
Q^{\pi_{\theta}}(s_t, a_t): S \rightarrow {\rm I\!R}.\\
V^{\pi_{\theta}}(s_t): S \rightarrow {\rm I\!R}.\\
Q^{\pi_{\theta}}(s_t, a_t) = \mathbb{E}_{\tau \sim \pi_{\theta}}[\mathcal{R}(\tau) | s_0 = s_t, a_0 = a_t] = \frac{1}{N}\sum_{n=1}^N \mathcal{R}_t^n.\\
V^{\pi_{\theta}}(s_t) = \mathbb{E}_{\tau \sim \pi_{\theta}}[\mathcal{R}(\tau) | s_0 = s_t].\\
V^{\pi_{\theta}}(s_t) = \mathbb{E}_{a_t \sim \pi_{\theta}(s_t)}[Q^{\pi}(s_t, a_t)].
\end{eqnarray}

To reduce variance even further one can introduce discount factors. However, if the time horizon is too long, i.e, if there are too many time steps in one episode, VPG will likely not work well. Therefore, the discount factor should be think off as a variance reduction parameter. 

---
**Algorithm (Pseudocode): Vanilla Policy Gradient Algorithm (adapted from Open AI)**

---

- Input: initialize policy parameters $\theta_0$, and baseline parameters $\phi_0$.

- for $k= 0, 1, 2, \dots$ do:

    - Collect a set of trajectories $\mathcal{H}_t \doteq \mathcal{D}_k\doteq\{\tau_i\} = (s_0, a_0, r_0, \cdots , s_N, a_{N}, r_N)$ by executing the current policy $\pi_k = \pi(\theta_k)$ in the environment.
    - For each trajectory, compute: 
        - the reward-to-go $\hat{\mathcal{R}}_t = \sum_{t'=t}^T R(s_t', a_t', s_{t'+1})$, and
        - the advantage estimates $\hat{A}_1, \dots, \hat{A}_T$ (using any advantage estimation method) based on the current  on-policy state value function $V_{\phi_k}$ used as the baseline: $$\hat{A}_t = Q^{\pi_{\theta}}(s_t, a_t) - V^{\pi_{\theta}}(s_t) \in {\rm I\!R}.$$   
    - Estimate the Policy gradient as:
$$\hat{g}_k = \frac{1}{|\mathcal{D}_k|}\sum_{\tau \in \mathcal{D}_k}\sum_{t=0}^T\nabla_{\theta}log\pi_{\theta}(a_t| s_t)|_{\theta_k}\hat{A}_t .$$
    - Update the Policy either using standard gradient ascent or via another `gradient ascent` algorithm (such as Adam): $$\theta_{k+1}= \theta_k + a_k \hat{g}_k.$$
    - Re-fit the baseline (value function) by regression on mean-squared error, via some `gradient descent` algorithm, by minimizing $(V_{\phi}(s_t) - \hat{\mathcal{R}}_t)^2$ summed over all trajectories and time steps: $$\phi_{k+1} = \underset{\phi}{\operatorname{arg\,min}} \ \frac{1}{|\mathcal{D}_k|T}\sum_{\tau \in \mathcal{D}_k} \sum_{t=0}^T \left(V_{\phi}(s_t) - \hat{\mathcal{R}}_t \right)^2 .$$ 

    
- end for.

---

# References

[1] https://spinningup.openai.com/en/latest/algorithms/vpg.html#quick-facts